<h1>Importando bibliotecas</h1>

In [75]:
import pandas as pd
import numpy as np
import sys
import sklearn
from scipy.spatial import distance
import pandas as pd
from sklearn.preprocessing import Normalizer
from sklearn.metrics import log_loss
import _pickle as cPickle
from IPython.display import clear_output
from scipy.optimize import minimize, rosen, rosen_der
from timeit import default_timer as timer
from datetime import timedelta

<h1>Variáveis auxiliares</h1>

In [2]:
#variáveis inteiras
col_namesInt = ['duration', 'src_bytes', 'dst_bytes', 'land',  'logged_in',  'is_host_login', 'is_guest_login',
'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'num_compromised',
 'root_shell',
 'su_attempted',
 'num_root',
 'num_file_creations',
 'num_shells',
 'num_access_files',
 'num_outbound_cmds',
 'count',
 'srv_count',
 'dst_host_count',
 'dst_host_srv_count']

#variáveis contínuas
col_namesFloat = [
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate',
]
#variáveis categoricas => one_hot_encoder => variáveis inteiras
categ = ['protocol_type_icmp',
 'protocol_type_tcp',
 'protocol_type_udp', 'service_IRC',
 'service_X11',
 'service_Z39_50',
 'service_aol',
 'service_auth',
 'service_bgp',
 'service_courier',
 'service_csnet_ns',
 'service_ctf',
 'service_daytime',
 'service_discard',
 'service_domain',
 'service_domain_u',
 'service_echo',
 'service_eco_i',
 'service_ecr_i',
 'service_efs',
 'service_exec',
 'service_finger',
 'service_ftp',
 'service_ftp_data',
 'service_gopher',
 'service_harvest',
 'service_hostnames',
 'service_http',
 'service_http_2784',
 'service_http_443',
 'service_http_8001',
 'service_imap4',
 'service_iso_tsap',
 'service_klogin',
 'service_kshell',
 'service_ldap',
 'service_link',
 'service_login',
 'service_mtp',
 'service_name',
 'service_netbios_dgm',
 'service_netbios_ns',
 'service_netbios_ssn',
 'service_netstat',
 'service_nnsp',
 'service_nntp',
 'service_ntp_u',
 'service_other',
 'service_pm_dump',
 'service_pop_2',
 'service_pop_3',
 'service_printer',
 'service_private',
 'service_red_i',
 'service_remote_job',
 'service_rje',
 'service_shell',
 'service_smtp',
 'service_sql_net',
 'service_ssh',
 'service_sunrpc',
 'service_supdup',
 'service_systat',
 'service_telnet',
 'service_tftp_u',
 'service_tim_i',
 'service_time',
 'service_urh_i',
 'service_urp_i',
 'service_uucp',
 'service_uucp_path',
 'service_vmnet',
 'service_whois','flag_OTH',
 'flag_REJ',
 'flag_RSTO',
 'flag_RSTOS0',
 'flag_RSTR',
 'flag_S0',
 'flag_S1',
 'flag_S2',
 'flag_S3',
 'flag_SF',
 'flag_SH']

In [3]:
tipoInteiro = {}
for s in col_namesInt:
    tipoInteiro[s] = True
for s in col_namesFloat:
    tipoInteiro[s] = False
for s in categ:
    tipoInteiro[s] = True
tipoInteiro

{'duration': True,
 'src_bytes': True,
 'dst_bytes': True,
 'land': True,
 'logged_in': True,
 'is_host_login': True,
 'is_guest_login': True,
 'wrong_fragment': True,
 'urgent': True,
 'hot': True,
 'num_failed_logins': True,
 'num_compromised': True,
 'root_shell': True,
 'su_attempted': True,
 'num_root': True,
 'num_file_creations': True,
 'num_shells': True,
 'num_access_files': True,
 'num_outbound_cmds': True,
 'count': True,
 'srv_count': True,
 'dst_host_count': True,
 'dst_host_srv_count': True,
 'serror_rate': False,
 'srv_serror_rate': False,
 'rerror_rate': False,
 'srv_rerror_rate': False,
 'same_srv_rate': False,
 'diff_srv_rate': False,
 'srv_diff_host_rate': False,
 'dst_host_same_srv_rate': False,
 'dst_host_diff_srv_rate': False,
 'dst_host_same_src_port_rate': False,
 'dst_host_srv_diff_host_rate': False,
 'dst_host_serror_rate': False,
 'dst_host_srv_serror_rate': False,
 'dst_host_rerror_rate': False,
 'dst_host_srv_rerror_rate': False,
 'protocol_type_icmp': True

In [4]:
df = pd.read_csv('datasets_adv/DoS_orig_normal.csv') #apenas auxiliar
df = df.drop('y_actual', 1)
df = df.drop('y_pred', 1)
colunas = df.columns
mapInd = {}
i = 0
for s in df.columns:
    mapInd[s] = i
    i += 1
mapInd

{'duration': 0,
 'src_bytes': 1,
 'dst_bytes': 2,
 'land': 3,
 'wrong_fragment': 4,
 'urgent': 5,
 'hot': 6,
 'num_failed_logins': 7,
 'logged_in': 8,
 'num_compromised': 9,
 'root_shell': 10,
 'su_attempted': 11,
 'num_root': 12,
 'num_file_creations': 13,
 'num_shells': 14,
 'num_access_files': 15,
 'num_outbound_cmds': 16,
 'is_host_login': 17,
 'is_guest_login': 18,
 'count': 19,
 'srv_count': 20,
 'serror_rate': 21,
 'srv_serror_rate': 22,
 'rerror_rate': 23,
 'srv_rerror_rate': 24,
 'same_srv_rate': 25,
 'diff_srv_rate': 26,
 'srv_diff_host_rate': 27,
 'dst_host_count': 28,
 'dst_host_srv_count': 29,
 'dst_host_same_srv_rate': 30,
 'dst_host_diff_srv_rate': 31,
 'dst_host_same_src_port_rate': 32,
 'dst_host_srv_diff_host_rate': 33,
 'dst_host_serror_rate': 34,
 'dst_host_srv_serror_rate': 35,
 'dst_host_rerror_rate': 36,
 'dst_host_srv_rerror_rate': 37,
 'protocol_type_icmp': 38,
 'protocol_type_tcp': 39,
 'protocol_type_udp': 40,
 'service_IRC': 41,
 'service_X11': 42,
 'service

<h1>Funções</h1>

In [5]:
def logloss(true_label, predicted, eps=1e-15):
 p = np.clip(predicted, eps, 1 - eps)
 if true_label == 1:
   return -np.log(p)
 else:
   return -np.log(1 - p)

In [6]:
def arredondarInteiros(X):
    X2 = np.array(X).reshape(1, -1)    
    for col in colunas: #dfDoS = global
        if tipoInteiro[col]:
            X2[0][mapInd[col]] = round(X2[0][mapInd[col]], 0)
    return X2

In [7]:
def obterDataset(filename, filenameOrig):
    df = pd.read_csv(filename)
    dfAux = pd.read_csv(filenameOrig)
    df = df.drop('y_actual', 1)
    df = df.drop('y_pred', 1)
    minimos = dfAux.min()
    maximos = dfAux.max()
    return (df, minimos, maximos)

In [8]:
def obterClassificador(filename):
    with open(filename, 'rb') as fid:
        model = cPickle.load(fid)
    return model

In [9]:
def obterScaler(filename):
    with open(filename, 'rb') as fid:
        scaler = cPickle.load(fid) 
    return scaler

In [53]:
def normalizar(df, scaler):
    df = df.drop('y_actual', 1)
    df = df.drop('y_pred', 1)
    X = scaler.transform(df)
    newDf = pd.DataFrame(X, columns = colunas.tolist())
    return newDf

In [11]:
def distancias(df1, df2):
    dists = []
    for i, row in df1.iterrows():
        X1 = []
        X2 = []
        for c in df1.columns:
            X1.append(df1[c][i])
            X2.append(df2[c][i])
        dists.append(distance.euclidean(X1, X2))
    return dists

In [12]:
def diferenca(df1, df2):
    dfRes = df1.copy()
    for i, row in df1.iterrows():
        for c in df1.columns:
            dfRes.at[i, c] = df2[c][i] - df1[c][i];
    return dfRes

In [13]:
model = scaler = y0 = X0 = None

In [77]:
def funcaoObjetivoInt(X):
    Xi = np.array(X0).reshape(1, -1)
    Xi = scaler.transform(Xi)
    X2 = arredondarInteiros(X)
    X2 = scaler.transform(X2)
    
    y = model.predict_proba(X2)[0][1]
    
    return 100*logloss(y0, y) + distance.euclidean(Xi, X2)

<h1>Otimização</h1>

In [70]:
def otimizar(filenameModel, filenameDf, filenameDfOrig, filenameScaler, classeReal, debugFlag=False):
    global scaler, model, X0, y0
    start = timer()
    df, minimos, maximos = obterDataset(filenameDf, filenameDfOrig)
    dfRes = pd.read_csv(filenameDf)
    scaler = obterScaler(filenameScaler)
    model = obterClassificador(filenameModel)
    y0 = classeReal
    dists = []
    
    bnds = []
    #definindo restrições
    for c in df.columns:
        bnds.append((minimos[c], maximos[c]))
    bnds = tuple(bnds)
    
    for i, row in df.iterrows():
        clear_output(wait=True)
        print(i,'/',df.shape[0],sep='', flush=True)
        X = []
        X0 = []
        for c in df.columns:
            X0.append(df[c][i])
            X.append(df[c][i])
        res = minimize(fun = funcaoObjetivoInt, x0 = X, method='Powell', options={'disp': debugFlag})
        Xaux = arredondarInteiros(res.x)
        classe = model.predict(scaler.transform(Xaux))
        j = 0
        for c in df.columns:
            dfRes.at[i, c] = Xaux[0][j]
            j += 1
        dfRes.at[i, 'y_pred'] = classe
        dists.append(distance.euclidean(X0, Xaux))
    clear_output(wait=True)
    print('Completo!', flush=True)
    end = timer()
    print(timedelta(seconds=end-start))
    return (dfRes, dists)

In [71]:
def otimizarAmostra(filenameModel, filenameDf, filenameDfOrig, filenameScaler, classeReal, i, debugFlag=False):
    global scaler, model, X0, y0
    start = timer()
    df, minimos, maximos = obterDataset(filenameDf, filenameDfOrig)
    dfRes = pd.read_csv(filenameDf)
    scaler = obterScaler(filenameScaler)
    model = obterClassificador(filenameModel)
    y0 = classeReal
    X = []
    X0 = []
    
    bnds = []
    #definindo restrições
    for c in df.columns:
        bnds.append((minimos[c], maximos[c]))
    bnds = tuple(bnds)
    
    i = 0
    #instanciando variáveis X (amostra adversária) e ponto inicial X0 (amostra original)
    for c in df.columns:
        X0.append(df[c][i])
        X.append(df[c][i])
    #Minimizando a função objetivo
    res = minimize(fun = funcaoObjetivoInt, x0 = X, method='Powell', options={'disp': debugFlag})
    Xaux = arredondarInteiros(res.x)
    classe = model.predict(scaler.transform(Xaux))
    end = timer()
    print(timedelta(seconds=end-start))
    return (Xaux, classe, distance.euclidean(X0, Xaux))

<h1>Aplicando para os datasets</h1>

<h2>Amostrais normais classificadas como DoS<h2>

In [46]:
fileClass = 'classificadores/rfc_DoS.pkl'
fileData = 'datasets_adv/DoS_orig_normal.csv'
fileDataOrig = 'DoS.csv'
fileScaler = 'scalers/scalerDoS.pkl'
filepathRes = 'datasets_resultados/Normal_DoS/'
classeReal = 0

In [47]:
#otimizarAmostra(fileClass, fileData, fileDataOrig, fileScaler, classeReal, 0, debugFlag = True)

In [48]:
dfRes, dists = otimizar(fileClass, fileData, fileDataOrig, fileScaler, classeReal, debugFlag = True)
dfRes

Completo!


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,1.0,1481.0,1.0,-0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,-0.0,28.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,-0.0,-0.0,0.0,0.0,-0.0,0,0
2,-0.0,1480.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,2.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0,0
4,1.0,1480.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
5,1.0,1481.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0,0
6,1.0,1481.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0,0
7,1.0,1480.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0,0
8,1.0,29.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [49]:
dfRes['y_pred'].value_counts()

0    9
Name: y_pred, dtype: int64

In [50]:
dists

[3.0000291665645307,
 3.3166323281470653,
 4.242646579734206,
 4.472135954999582,
 6.782333669179468,
 3.464108832318343,
 3.6058078150524184,
 3.872993028660224,
 3.464108832030556]

In [51]:
dfOrig = pd.read_csv(fileData)
scaler = obterScaler(fileScaler)
dfOrig

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1,0.0,28.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
2,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
4,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
5,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
6,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
7,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
8,0.0,28.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1


In [54]:
dfResNorm = normalizar(dfRes, scaler)

In [55]:
dfOrigNorm = normalizar(dfOrig, scaler)

In [56]:
distancias(dfResNorm, dfOrigNorm)

[0.0741796823850771,
 0.053302324872760966,
 0.04385616390460414,
 0.04840517235660893,
 0.07121741924818425,
 0.023799999543047245,
 0.11366800495409858,
 0.04150722078885812,
 0.04490225587840072]

In [57]:
dfDif = diferenca(dfOrig, dfRes)
dfDif

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,1.0,1.0,1.0,-0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1
1,-0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,-0.0,0,-1
2,-0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1
3,2.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0,-1
4,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1
5,1.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0,-1
6,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0,-1
7,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0,-1
8,1.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1


In [58]:
#exportando datasets
dfOrig.to_csv(filepathRes + 'orig.csv', index=False)
dfOrigNorm.to_csv(filepathRes + 'orig_normalizado.csv', index=False)
dfRes.to_csv(filepathRes + 'advers.csv', index=False)
dfResNorm.to_csv(filepathRes + 'advers_normalizado.csv', index=False)
dfDif.to_csv(filepathRes + 'diferencas.csv', index=False)

<h2>Amostrais DoS classificadas como normais<h2>

In [78]:
fileClass = 'classificadores/rfc_DoS.pkl'
fileData = 'datasets_adv/DoS_orig_ataques.csv'
fileDataOrig = 'DoS.csv'
fileScaler = 'scalers/scalerDoS.pkl'
filepathRes = 'datasets_resultados/DoS_Normal/'
classeReal = 1

In [79]:
otimizarAmostra(fileClass, fileData, fileDataOrig, fileScaler, classeReal, 1, debugFlag = True)

Optimization terminated successfully.
         Current function value: 4.625389
         Iterations: 5
         Function evaluations: 29207
0:05:00.219937


(array([[-0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  0.00000000e+00, -0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  2.20000000e+01,  2.00000000e+00,
          9.99999999e-01,  9.99999998e-01,  4.99999897e-03,
         -9.43891863e-09,  1.94999844e-01,  3.49999999e-02,
          9.99999999e-01,  2.55000000e+02,  2.00000000e+00,
          9.09556036e-12,  4.50000015e-02,  2.76181385e-10,
          6.93388616e-11,  9.89999997e-01,  9.99999999e-01,
          3.50000087e-02, -2.58379662e-09,  0.00000000e+00,
          1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+0

In [80]:
dfRes, dists = otimizar(fileClass, fileData, fileDataOrig, fileScaler, classeReal, debugFlag = True)
dfRes

Completo!
1:50:13.357997


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,-0.0,-0.0,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,-0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,0.0,564.0,0.0,0.0,1.0,-0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1
2,0.0,28.0,0.0,-0.0,1.0,0.0,0.0,0.0,0.0,-0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,1
3,0.0,28.0,0.0,0.0,2.0,0.0,-0.0,0.0,0.0,-0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,1
4,3.0,35.0,255.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,...,1.0,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0,1,1
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.0,0.0,...,1.0,-0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,1
6,0.0,0.0,0.0,1.0,-0.0,0.0,-0.0,0.0,0.0,0.0,...,0.0,-0.0,1.0,0.0,0.0,0.0,-0.0,0.0,1,1
7,0.0,28.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.0,1,1
8,0.0,28.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.0,1,1
9,-0.0,28.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.0,0.0,0.0,0.0,1.0,0.0,1,1


In [81]:
dfRes['y_pred'].value_counts()

1    28
Name: y_pred, dtype: int64

In [82]:
dists

[21.062555050287443,
 2.592388640858255,
 6.900913049704748,
 2.792539180619424,
 3.0388692678605653,
 4.354962766971916,
 41.09145714146338,
 2.8108547013174174,
 2.7875303969897005,
 2.0000000000001137,
 2.8692813353524524,
 3.0129263368655024,
 2.812074357928982,
 3.545800436256563,
 3.9457736738499154,
 2.1354862250665194,
 3.335468491253398,
 3.2250545193594164,
 8.162174972295922,
 3.902137560732828,
 4.051743462832522,
 3.1046497593402966,
 3.9635180825508023,
 7.66918386504806,
 5.937697369140854,
 50.26661019046209,
 3.4820719390411443,
 2.3130499305812102]

In [83]:
dfOrig = pd.read_csv(fileData)
scaler = obterScaler(fileScaler)
dfOrig

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1,0.0,564.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2,0.0,28.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3,0.0,28.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
4,3.0,35.0,255.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
7,0.0,28.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
8,0.0,28.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
9,0.0,28.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0


In [84]:
dfResNorm = normalizar(dfRes, scaler)

In [85]:
dfOrigNorm = normalizar(dfOrig, scaler)

In [86]:
distancias(dfResNorm, dfOrigNorm)

[2.8598954590235013,
 10.57697728172999,
 5.822581713891607,
 6.922810631374053,
 6.276452555556258,
 8.84863218296443,
 4.3904269557905655,
 7.109986385402537,
 5.540745347183751,
 3.7449984753961223,
 5.32753037262041,
 6.552285276398605,
 10.114908388948868,
 3.52684697835255,
 4.502640605464275,
 8.814527566926547,
 5.951794675336848,
 3.3734200070477978,
 3.8996961049990677,
 6.334736624637241,
 3.0683334227400305,
 8.424419907733503,
 5.809295571939646,
 2.266957980374252,
 3.0425454335154214,
 1.9422804031518028,
 8.102375465644787,
 4.100680273994947]

In [87]:
dfDif = diferenca(dfOrig, dfRes)
dfDif

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,-0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1,0.0,0.0,0.0,0.0,1.0,-0.0,0.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1
2,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0,...,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,1
3,0.0,0.0,0.0,0.0,-1.0,0.0,-0.0,0.0,0.0,-0.0,...,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,1
4,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,...,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0,1
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.0,0.0,...,0.0,-0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,1
6,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,...,0.0,-0.0,1.0,0.0,0.0,0.0,-0.0,0.0,0,1
7,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.0,0,1
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,...,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.0,0,1
9,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,0.0,-0.0,0.0,0.0,0.0,1.0,0.0,0,1


In [88]:
#exportando datasets
dfOrig.to_csv(filepathRes + 'orig.csv', index=False)
dfOrigNorm.to_csv(filepathRes + 'orig_normalizado.csv', index=False)
dfRes.to_csv(filepathRes + 'advers.csv', index=False)
dfResNorm.to_csv(filepathRes + 'advers_normalizado.csv', index=False)
dfDif.to_csv(filepathRes + 'diferencas.csv', index=False)

<h2>Amostrais normais classificadas como Probe<h2>

In [89]:
fileClass = 'classificadores/rfc_Probe.pkl'
fileData = 'datasets_adv/Probe_orig_normal.csv'
fileDataOrig = 'Probe.csv'
fileScaler = 'scalers/scalerProbe.pkl'
filepathRes = 'datasets_resultados/Normal_Probe/'
classeReal = 0

In [90]:
#otimizarAmostra(fileClass, fileData, fileDataOrig, fileScaler, classeReal, 0, debugFlag = True)

In [91]:
dfRes, dists = otimizar(fileClass, fileData, fileDataOrig, fileScaler, classeReal, debugFlag = True)
dfRes

Completo!
1:14:08.941025


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,1.0,-0.0,...,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,0,0
1,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,1.0,-0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0,0
2,0.0,3.0,1.0,0.0,-0.0,-0.0,0.0,0.0,1.0,-0.0,...,1.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0,0
3,-0.0,1.0,1.0,0.0,0.0,0.0,-0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,0.0,1.0,3.0,0.0,0.0,-0.0,0.0,0.0,1.0,-0.0,...,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0,0
5,0.0,0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,1.0,-0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
6,0.0,1.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,1.0,-0.0,...,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,0,0
7,0.0,1.0,2.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0,0
8,-0.0,-0.0,4.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0,0
9,3.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.0,...,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0,0


In [92]:
dfRes['y_pred'].value_counts()

0    19
Name: y_pred, dtype: int64

In [93]:
dists

[3.385982998291896,
 2.607547392713053,
 3.793537297662454,
 6.160935408348553,
 4.32940820304037,
 2.168273677215128,
 4.629605814516827,
 2.853002809208373,
 6.3145981994442355,
 3.8731678304052504,
 5.304936385967234,
 2.056181686711053,
 3.4917372270312157,
 3.9879415263196463,
 2.0754221790447414,
 4.0251118061656035,
 4.496270797264411,
 3.6016491398060424,
 4.116045049918587]

In [94]:
dfOrig = pd.read_csv(fileData)
scaler = obterScaler(fileScaler)
dfOrig

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,-5.684342e-14,0.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1,-5.684342e-14,0.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
2,-5.684342e-14,1.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
3,-5.684342e-14,1.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
4,-5.684342e-14,0.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
5,-5.684342e-14,0.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
6,-5.684342e-14,0.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
7,-5.684342e-14,1.0,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
8,3.000000e+00,0.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
9,3.000000e+00,0.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1


In [95]:
dfResNorm = normalizar(dfRes, scaler)

In [96]:
dfOrigNorm = normalizar(dfOrig, scaler)

In [97]:
distancias(dfResNorm, dfOrigNorm)

[8.527400083458332,
 7.733735700723853,
 8.512814604710066,
 4.346180991452413,
 9.518665903718203,
 6.532043832891467,
 4.766780309303223,
 1.5354155332529498,
 8.764203225970084,
 8.084036491380104,
 3.977886520609034,
 3.947456924696788,
 9.174134888441523,
 13.259143537280785,
 5.066540773371748,
 4.813345336372503,
 8.71329764135551,
 6.56200750288728,
 14.578865129521432]

In [98]:
dfDif = diferenca(dfOrig, dfRes)
dfDif

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,5.684342e-14,0.0,-3.637979e-12,0.0,-0.0,-0.0,-0.0,-0.0,1.0,-0.0,...,2.168404e-19,-0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,0,-1
1,5.684342e-14,0.0,-3.637979e-12,0.0,-0.0,-0.0,-0.0,0.0,1.0,-0.0,...,2.168404e-19,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0,-1
2,5.684342e-14,2.0,1.000000e+00,0.0,-0.0,-0.0,0.0,0.0,1.0,-0.0,...,0.000000e+00,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0,-1
3,5.684342e-14,0.0,1.000000e+00,0.0,0.0,0.0,-0.0,0.0,1.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1
4,5.684342e-14,1.0,3.000000e+00,0.0,0.0,-0.0,0.0,0.0,1.0,-0.0,...,2.168404e-19,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0,-1
5,5.684342e-14,0.0,-3.637979e-12,0.0,-0.0,0.0,-0.0,-0.0,1.0,-0.0,...,2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1
6,5.684342e-14,1.0,-3.637979e-12,0.0,-0.0,0.0,-0.0,-0.0,1.0,-0.0,...,2.168404e-19,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,0,-1
7,5.684342e-14,0.0,1.000000e+00,0.0,-0.0,-0.0,-0.0,0.0,0.0,1.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0,-1
8,-3.000000e+00,-0.0,4.000000e+00,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,...,2.168404e-19,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0,-1
9,-5.684342e-14,-0.0,-3.637979e-12,0.0,0.0,0.0,0.0,0.0,1.0,-0.0,...,2.168404e-19,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0,-1


In [99]:
#exportando datasets
dfOrig.to_csv(filepathRes + 'orig.csv', index=False)
dfOrigNorm.to_csv(filepathRes + 'orig_normalizado.csv', index=False)
dfRes.to_csv(filepathRes + 'advers.csv', index=False)
dfResNorm.to_csv(filepathRes + 'advers_normalizado.csv', index=False)
dfDif.to_csv(filepathRes + 'diferencas.csv', index=False)

<h2>Amostrais Probe classificadas como normais<h2>

In [100]:
fileClass = 'classificadores/rfc_Probe.pkl'
fileData = 'datasets_adv/Probe_orig_ataques.csv'
fileDataOrig = 'Probe.csv'
fileScaler = 'scalers/scalerProbe.pkl'
filepathRes = 'datasets_resultados/Probe_Normal/'
classeReal = 1

In [101]:
otimizarAmostra(fileClass, fileData, fileDataOrig, fileScaler, classeReal, 0, debugFlag = True)

Optimization terminated successfully.
         Current function value: 25.361203
         Iterations: 4
         Function evaluations: 22210
0:03:14.887497


(array([[ 0.00000000e+00,  3.90000000e+01,  4.00000000e+01,
          0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
          1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  3.00000000e+00,  2.00000000e+00,
         -2.87009189e-10,  6.00000118e-02,  5.50000051e-01,
          9.25000044e-01,  2.25000014e-01,  7.50000040e-01,
          3.92115955e-11,  7.00000000e+01,  1.00000000e+00,
          9.99999781e-03,  4.14999991e-01,  9.99997465e-03,
         -1.83505977e-08,  9.50006683e-02, -8.84785230e-09,
          6.99999987e-02,  5.75000018e-01,  0.00000000e+00,
          1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -0.00000000e+00,  0.00000000e+0

In [102]:
dfRes, dists = otimizar(fileClass, fileData, fileDataOrig, fileScaler, classeReal, debugFlag = True)
dfRes

Completo!
1:52:08.405156


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,0.0,39.0,40.0,0.0,0.0,-0.0,1.0,0.0,0.0,0.0,...,1.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,-0.0,0.0,1.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,...,1.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,1,1
2,1.0,1.0,1.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
3,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,-0.0,0.0,0.0,1.0,-0.0,1,1
4,1.0,3.0,25.0,0.0,-0.0,-0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
5,-0.0,1.0,1.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,1.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,1,1
6,1.0,5.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,-0.0,...,1.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,1,1
7,1.0,6.0,44.0,0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,...,1.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,1,1
8,2.0,24.0,942.0,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,1,1
9,-0.0,1.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,1,1


In [103]:
dfRes['y_pred'].value_counts()

1    31
Name: y_pred, dtype: int64

In [104]:
dists

[3.700692546311509,
 3.0307961059355315,
 2.791182554350579,
 2.2721521266003233,
 3.7647346846295817,
 2.061164723873773,
 2.9655314908944663,
 3.6321550228822232,
 2.322660574437693,
 2.533762125637366,
 3.1166207672070736,
 5.5294371421049515,
 2.655089467396926,
 2.7790646243036887,
 2.549912075527404,
 2.467240381627582,
 2.2325550849132356,
 2.6849069091294164,
 4.342418868167037,
 2.5361537815878843,
 1.8572695012888993,
 2.550500146679704,
 2.3181997867013338,
 2.1031761652774343,
 3.9395083003698947,
 5.411395423359997,
 2.2818194501023177,
 3.9991405333116967,
 5.577847275489831,
 3.4405740963652782,
 2.665576910272792]

In [105]:
dfOrig = pd.read_csv(fileData)
scaler = obterScaler(fileScaler)
dfOrig

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,-5.684342e-14,40.0,4.000000e+01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1,-5.684342e-14,1.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2,-5.684342e-14,0.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3,-5.684342e-14,1.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
4,1.000000e+00,6.0,2.500000e+01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
5,-5.684342e-14,1.0,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
6,-5.684342e-14,5.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
7,-5.684342e-14,9.0,4.400000e+01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
8,2.000000e+00,24.0,9.420000e+02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
9,-5.684342e-14,0.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0


In [106]:
dfResNorm = normalizar(dfRes, scaler)

In [107]:
dfOrigNorm = normalizar(dfOrig, scaler)

In [108]:
distancias(dfResNorm, dfOrigNorm)

[9.109310154375812,
 4.102579993475355,
 2.956225627927445,
 5.937181750560242,
 5.42694144951492,
 4.934157659998698,
 9.408071646664807,
 6.95112929734467,
 4.896287168958538,
 7.550438574985876,
 5.48827841380508,
 6.619111337374287,
 4.123741083809388,
 6.3905339175845555,
 6.609975097233174,
 8.536216547884608,
 5.82711152288712,
 7.01017222242918,
 3.060109134816298,
 9.53981888967669,
 12.089688911636767,
 4.553800766891481,
 7.198032154058277,
 7.702964523131496,
 6.574034975052241,
 5.9389227960421715,
 4.845804844350272,
 8.347072531624475,
 5.398019361741127,
 8.488137865470234,
 6.7056800168741315]

In [109]:
dfDif = diferenca(dfOrig, dfRes)
dfDif

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,5.684342e-14,-1.0,-3.637979e-12,0.0,0.0,-0.0,1.0,0.0,0.0,0.0,...,0.000000e+00,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1,5.684342e-14,-1.0,1.000000e+00,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,...,0.000000e+00,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0,1
2,1.000000e+00,1.0,1.000000e+00,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
3,5.684342e-14,-1.0,-3.637979e-12,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,-0.0,0.0,0.0,1.0,-0.0,0,1
4,0.000000e+00,-3.0,0.000000e+00,0.0,-0.0,-0.0,1.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
5,5.684342e-14,0.0,0.000000e+00,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0,1
6,1.000000e+00,0.0,-3.637979e-12,0.0,-0.0,0.0,-0.0,0.0,0.0,-0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0,1
7,1.000000e+00,-3.0,0.000000e+00,0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,...,0.000000e+00,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0,1
8,0.000000e+00,0.0,3.637979e-12,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0,1
9,5.684342e-14,1.0,-3.637979e-12,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,...,2.168404e-19,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0,1


In [110]:
#exportando datasets
dfOrig.to_csv(filepathRes + 'orig.csv', index=False)
dfOrigNorm.to_csv(filepathRes + 'orig_normalizado.csv', index=False)
dfRes.to_csv(filepathRes + 'advers.csv', index=False)
dfResNorm.to_csv(filepathRes + 'advers_normalizado.csv', index=False)
dfDif.to_csv(filepathRes + 'diferencas.csv', index=False)

<h2>Amostrais normais classificadas como R2L<h2>

In [111]:
fileClass = 'classificadores/rfc_R2L.pkl'
fileData = 'datasets_adv/R2L_orig_normal.csv'
fileDataOrig = 'R2L.csv'
fileScaler = 'scalers/scalerR2L.pkl'
filepathRes = 'datasets_resultados/Normal_R2L/'
classeReal = 0

In [112]:
#otimizarAmostra(fileClass, fileData, fileDataOrig, fileScaler, classeReal, 0, debugFlag = True)

In [113]:
dfRes, dists = otimizar(fileClass, fileData, fileDataOrig, fileScaler, classeReal, debugFlag = True)
dfRes

Completo!
5:10:47.280515


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,1.0,105.0,146.0,0.0,-0.0,0.0,2.0,0.0,-0.0,0.0,...,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,-0.0,687.0,1.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,...,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,-0.0,105.0,147.0,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,...,-0.0,0.0,0.0,0.0,0.0,-0.0,1.0,0.0,0,0
3,-0.0,105.0,146.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,-0.0,106.0,146.0,0.0,0.0,0.0,4.0,0.0,-0.0,0.0,...,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,-0.0,105.0,146.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
86,60.0,129.0,179.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.0,...,0.0,-0.0,-0.0,0.0,0.0,0.0,1.0,0.0,0,0
87,28.0,6291668.0,1.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0,0
88,-0.0,106.0,146.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0,0


In [114]:
dfRes['y_pred'].value_counts()

0    90
Name: y_pred, dtype: int64

In [115]:
dists

[3.512314055459618,
 4.502185586838633,
 2.0000000000000027,
 6.8734125439188904,
 4.709976114936724,
 1.732086947102162,
 1.7415079678463248,
 1.7414433683111443,
 4.5675540531902765,
 2.00206768895234,
 4.633435765899134,
 4.130686992667184,
 3.740327591905303,
 2.23606797749979,
 2.000018750613406,
 4.131561448519593,
 2.0461916503337427,
 4.366068026596632,
 4.582578422679969,
 1.7320580253667552,
 4.030356705128356,
 4.697084202764528,
 4.123190514530593,
 2.449749988858157,
 3.4731182094177337,
 5.102249520537439,
 2.6457513110645907,
 2.5049251419702236,
 4.14650455133254,
 5.662380245552192,
 3.6142184497331353,
 2.2774878011285162,
 1.7320508075689234,
 4.4722338001430355,
 2.236073567699445,
 2.000012500021713,
 2.4884382664849234,
 3.0285846540784203,
 1.414231242049413,
 2.9417681825637017,
 4.123190515629372,
 1.7320580244249204,
 3.0016745326412364,
 2.000006249992437,
 5.57337869206992,
 4.917608696051423,
 3.3446449794727546,
 2.6457607601544346,
 2.2724711232128434,
 3

In [116]:
dfOrig = pd.read_csv(fileData)
scaler = obterScaler(fileScaler)
dfOrig

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,0.0,105.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,0,1
1,0.0,687.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,0,1
2,0.0,105.0,147.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,0,1
3,0.0,105.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,0,1
4,0.0,105.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0.0,105.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,0,1
86,60.0,129.0,179.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,0,1
87,28.0,6291668.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,0,1
88,0.0,105.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,0,1


In [117]:
dfResNorm = normalizar(dfRes, scaler)

In [118]:
dfOrigNorm = normalizar(dfOrig, scaler)

In [119]:
distancias(dfResNorm, dfOrigNorm)

[4.487674820694829,
 4.756337471138819,
 4.914935636933829,
 2.3170306561581384,
 4.329692633338018,
 2.461469014053512,
 1.5464170414413285,
 2.9329063833068867,
 6.731050569511414,
 4.314545363844444,
 4.576786933467153,
 2.6549913573767046,
 4.488344638173239,
 1.4144751774083177,
 1.4163259037543625,
 3.592714290375054,
 3.0127761339922787,
 3.455704292950099,
 3.6895665452127293,
 1.414379606058177,
 2.3145805342458012,
 3.2353613899442872,
 4.205959078471918,
 9.09211894759751,
 1.7293897614584968,
 6.510916243356188,
 4.483774307018063,
 17.174858979155584,
 1.734698196604617,
 2.3864470198045065,
 1.6247921248017103,
 4.548407351648099,
 4.483774306974899,
 8.066316971319832,
 2.46032993406954,
 1.0031034523947888,
 3.770532427027647,
 4.634574727914213,
 20.218270398675376,
 8.473878931089235,
 4.2059591443429145,
 4.483812454509649,
 3.5053966116400423,
 2.4611951539518215,
 2.3864470197799346,
 3.459400894396499,
 1.6855842666634275,
 1.059888190901065,
 1.8870592832799435,


In [120]:
dfDif = diferenca(dfOrig, dfRes)
dfDif

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,1.0,0.0,0.0,0.0,-0.0,0.0,2.0,0.0,-0.0,0.0,...,-1.0,0.0,-0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,-1
1,-0.0,0.0,1.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,...,-1.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,-1
2,-0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,0.0,2.168404e-19,-0.0,1.0,0.0,0,-1
3,-0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-1.0,-0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,-1
4,-0.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,-0.0,0.0,...,-1.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,-1
86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.0,...,-1.0,-0.0,-0.0,0.0,2.168404e-19,0.0,1.0,0.0,0,-1
87,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,...,-1.0,0.0,0.0,0.0,2.168404e-19,-0.0,-0.0,0.0,0,-1
88,-0.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,2.168404e-19,0.0,-0.0,0.0,0,-1


In [121]:
#exportando datasets
dfOrig.to_csv(filepathRes + 'orig.csv', index=False)
dfOrigNorm.to_csv(filepathRes + 'orig_normalizado.csv', index=False)
dfRes.to_csv(filepathRes + 'advers.csv', index=False)
dfResNorm.to_csv(filepathRes + 'advers_normalizado.csv', index=False)
dfDif.to_csv(filepathRes + 'diferencas.csv', index=False)

<h2>Amostrais R2L classificadas como normais<h2>

In [122]:
fileClass = 'classificadores/rfc_R2L.pkl'
fileData = 'datasets_adv/R2L_orig_ataques.csv'
fileDataOrig = 'R2L.csv'
fileScaler = 'scalers/scalerR2L.pkl'
filepathRes = 'datasets_resultados/R2L_Normal/'
classeReal = 1

In [123]:
otimizarAmostra(fileClass, fileData, fileDataOrig, fileScaler, classeReal, 0, debugFlag = True)

Optimization terminated successfully.
         Current function value: 13.978090
         Iterations: 3
         Function evaluations: 16681
0:02:40.443126


(array([[-0.00000000e+00,  1.23000000e+02,  1.78000000e+02,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         -3.06577762e-11, -4.53156306e-10,  1.00000000e+00,
          1.00000000e+00,  9.99999999e-01, -2.28186958e-10,
          1.12586434e-10,  2.20000000e+01,  6.00000000e+00,
          7.75000164e-01,  1.49998973e-02,  4.99999766e-03,
         -4.52414253e-09,  5.00000285e-03, -6.80837487e-10,
          9.00000000e-02,  3.29999987e-01,  0.00000000e+00,
          1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+0

In [124]:
dfRes, dists = otimizar(fileClass, fileData, fileDataOrig, fileScaler, classeReal, debugFlag = True)
dfRes

Completo!
9:08:51.905686


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,-0.0,123.0,178.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,-0.0,12.0,35.0,0.0,0.0,-0.0,-0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
2,7.0,118.0,2425.0,-0.0,-0.0,-0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
3,218.0,181832.0,24172.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
4,9.0,146.0,146.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,0.0,105.0,147.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
145,0.0,12.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,...,1.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,1,1
146,-0.0,176412.0,1.0,0.0,-0.0,0.0,0.0,1.0,0.0,-0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
147,-0.0,105.0,147.0,0.0,-0.0,0.0,0.0,1.0,-0.0,0.0,...,1.0,-0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,1,1


In [125]:
dfRes['y_pred'].value_counts()

1    149
Name: y_pred, dtype: int64

In [126]:
dists

[4.274657903461817,
 2.880494750558442,
 4.219831162550636,
 2.0001687434820683,
 9.219547169043569,
 2.872320497664301,
 2.6287829993455194,
 3.150075395975536,
 2.9432084002913967,
 2.794780884950745,
 3.767820990357304,
 2.771069335596298,
 3.332428892595092,
 5.656909491911131,
 2.0084946075919072,
 2.389958170680228,
 2.3236931895431363,
 1.7321157582185835,
 2.765904158811698,
 2.709654038234763,
 4.4743323562207555,
 7.937586220244148,
 8.062335294261594,
 2.518387381618888,
 1.4142224012544033,
 3.004534783724074,
 4.533966334061408,
 1.7358643483639558,
 4.744496842670735,
 2.236073567648619,
 2.8285376087122063,
 4.584481977364212,
 2.3595288008181496,
 3.457755783266919,
 3.4946625497168093,
 2.3702005736066427,
 2.7330162774602202,
 3.4402361851760555,
 2.6109146248210715,
 4.767294853370395,
 4.401076810251671,
 2.697336663417107,
 2.3968053525561888,
 3.1497182391784264,
 2.50105479100584,
 2.7649729554438642,
 2.236487201408946,
 2.31133619447623,
 3.1711082533053765,
 3

In [127]:
dfOrig = pd.read_csv(fileData)
scaler = obterScaler(fileScaler)
dfOrig

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,4.0,123.0,178.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,1,0
1,0.0,10.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,1,0
2,10.0,118.0,2425.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,1,0
3,218.0,181832.0,24172.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,1,0
4,0.0,146.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,0.0,105.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,1,0
145,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,1,0
146,0.0,176412.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,1,0
147,0.0,105.0,147.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,1,0


In [128]:
dfResNorm = normalizar(dfRes, scaler)

In [129]:
dfOrigNorm = normalizar(dfOrig, scaler)

In [130]:
distancias(dfResNorm, dfOrigNorm)

[2.3247083672594213,
 21.743923236932037,
 18.916738612908375,
 1.7432523970449383,
 14.005576453599234,
 18.917031990969157,
 5.138142213759161,
 20.045447097576268,
 20.287190504885512,
 17.63301583098004,
 1.8298892903912032,
 18.906785533294382,
 14.556088238840283,
 18.769880249352944,
 2.0231282998104736,
 4.829493105690157,
 3.2930276438540393,
 0.052521871849479575,
 18.94087575863343,
 19.146769810160155,
 16.09112907149701,
 0.2795831847837199,
 1.0108672751594607,
 19.146802670689535,
 0.03133600957492103,
 18.544694613322317,
 17.34679016342498,
 8.093877200338765,
 8.753124384031521,
 0.026312505986876258,
 21.975721023095282,
 4.697198033751956,
 4.58255171868426,
 24.164075506683766,
 23.19651144895755,
 3.802689888363904,
 13.801839982384228,
 18.32864293101252,
 17.461215194582124,
 5.030840804146515,
 19.207389569393914,
 18.894969453550523,
 20.44162252415907,
 21.533758019781363,
 5.077459825658743,
 19.38146731635726,
 8.126514885477627,
 19.14680267068642,
 17.902

In [131]:
dfDif = diferenca(dfOrig, dfRes)
dfDif

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,...,1.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,1
1,-0.0,2.0,0.0,0.0,0.0,-0.0,-0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,1
2,-3.0,0.0,0.0,-0.0,-0.0,-0.0,1.0,1.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,1
3,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,1
4,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,1
145,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,...,0.0,0.0,0.0,-0.0,2.168404e-19,0.0,0.0,0.0,0,1
146,-0.0,0.0,1.0,0.0,-0.0,0.0,0.0,1.0,0.0,-0.0,...,1.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,1
147,-0.0,0.0,0.0,0.0,-0.0,0.0,0.0,1.0,-0.0,0.0,...,0.0,-0.0,-0.0,0.0,2.168404e-19,0.0,-0.0,0.0,0,1


In [132]:
#exportando datasets
dfOrig.to_csv(filepathRes + 'orig.csv', index=False)
dfOrigNorm.to_csv(filepathRes + 'orig_normalizado.csv', index=False)
dfRes.to_csv(filepathRes + 'advers.csv', index=False)
dfResNorm.to_csv(filepathRes + 'advers_normalizado.csv', index=False)
dfDif.to_csv(filepathRes + 'diferencas.csv', index=False)

<h2>Amostrais normais classificadas como U2R<h2>

In [133]:
fileClass = 'classificadores/rfc_U2R.pkl'
fileData = 'datasets_adv/U2R_orig_normal.csv'
fileDataOrig = 'U2R.csv'
fileScaler = 'scalers/scalerU2R.pkl'
filepathRes = 'datasets_resultados/Normal_U2R/'
classeReal = 0

In [134]:
#otimizarAmostra(fileClass, fileData, fileDataOrig, fileScaler, classeReal, 0, debugFlag = True)

In [135]:
dfRes, dists = otimizar(fileClass, fileData, fileDataOrig, fileScaler, classeReal, debugFlag = True)
dfRes

Completo!
0:00:00.994308


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred


In [136]:
dfRes['y_pred'].value_counts()

Series([], Name: y_pred, dtype: int64)

In [137]:
dists

[]

In [138]:
dfOrig = pd.read_csv(fileData)
scaler = obterScaler(fileScaler)
dfOrig

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred


In [139]:
dfResNorm = normalizar(dfRes, scaler)

ValueError: Found array with 0 sample(s) (shape=(0, 122)) while a minimum of 1 is required by StandardScaler.

In [ ]:
dfOrigNorm = normalizar(dfOrig, scaler)

In [ ]:
distancias(dfResNorm, dfOrigNorm)

In [ ]:
dfDif = diferenca(dfOrig, dfRes)
dfDif

In [ ]:
#exportando datasets
dfOrig.to_csv(filepathRes + 'orig.csv', index=False)
dfOrigNorm.to_csv(filepathRes + 'orig_normalizado.csv', index=False)
dfRes.to_csv(filepathRes + 'advers.csv', index=False)
dfResNorm.to_csv(filepathRes + 'advers_normalizado.csv', index=False)
dfDif.to_csv(filepathRes + 'diferencas.csv', index=False)

<h2>Amostrais U2R classificadas como normais<h2>

In [140]:
fileClass = 'classificadores/rfc_U2R.pkl'
fileData = 'datasets_adv/U2R_orig_ataques.csv'
fileDataOrig = 'U2R.csv'
fileScaler = 'scalers/scalerU2R.pkl'
filepathRes = 'datasets_resultados/U2R_Normal/'
classeReal = 1

In [141]:
otimizarAmostra(fileClass, fileData, fileDataOrig, fileScaler, classeReal, 0, debugFlag = True)

Optimization terminated successfully.
         Current function value: 41.125867
         Iterations: 5
         Function evaluations: 28781
0:05:01.300599


(array([[ 1.63000000e+02, -1.62000000e+05, -2.55698000e+05,
         -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
          3.00000000e+00, -0.00000000e+00,  1.00000000e+00,
          2.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -0.00000000e+00, -1.00000000e+00, -0.00000000e+00,
          2.73857251e-03,  9.97105176e-04,  3.76612457e-02,
          5.00000148e-03,  9.78772060e-01,  1.84798487e-02,
          5.00000076e-03,  1.93000000e+02, -4.20000000e+01,
          9.54999997e-01,  9.99999964e-03,  1.55000009e-01,
         -1.81579904e-03,  2.98463951e-03,  1.30135251e-03,
          5.00000076e-03,  3.44931547e-02,  0.00000000e+00,
          1.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+0

In [142]:
dfRes, dists = otimizar(fileClass, fileData, fileDataOrig, fileScaler, classeReal, debugFlag = True)
dfRes

Completo!
1:41:50.660913


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,163.0,-162000.0,-255698.0,-0.0,0.0,-0.0,3.0,-0.0,1.0,2.0,...,1.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.0,1,1
1,5601.0,-101707.0,2603796.0,-0.0,0.0,-0.0,3.0,1.0,1.0,14.0,...,1.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,1,1
2,85.0,-162035.0,-255155.0,-0.0,-0.0,0.0,3.0,1.0,1.0,3.0,...,1.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,1,1
3,163.0,-106885.0,2523255.0,-0.0,0.0,2.0,3.0,-0.0,1.0,23.0,...,1.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,1,1
4,31.0,-162567.0,-305589.0,-0.0,0.0,-0.0,3.0,-0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,1,1
5,75.0,-162372.0,-255232.0,-0.0,-0.0,-0.0,3.0,-0.0,1.0,1.0,...,1.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,1,1
6,93.0,-161858.0,-184468.0,-0.0,0.0,-0.0,3.0,0.0,1.0,1.0,...,1.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,1,1
7,105.0,-161273.0,-263612.0,-0.0,0.0,-0.0,3.0,-0.0,1.0,1.0,...,1.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,1,1
8,30.0,-125452.0,-305589.0,-0.0,0.0,-0.0,3.0,-0.0,1.0,1.0,...,1.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,1,1
9,210.0,-98285.0,1667029.0,-0.0,0.0,2.0,3.0,-0.0,1.0,1.0,...,1.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,1,1


In [143]:
dfRes['y_pred'].value_counts()

1    25
Name: y_pred, dtype: int64

In [144]:
dists

[3.2862496538036536,
 3.8051933935452134,
 4.826568659082573,
 3.279926127564421,
 6.114750978598706,
 4.044704107467582,
 4.0537895530219785,
 2.6677009926493795,
 5.321403752290434,
 4.797737759727366,
 3.2500277654597944,
 6.469958972343683,
 3.969853775880565,
 4.235054157520505,
 12.85205542749191,
 4.383338160971936,
 3.67603353886009,
 6.274363105528854,
 3.4132386684724265,
 3.5166678496895933,
 5.25426219367731,
 5.551185520213266,
 4.341616241896494,
 5.370586164574416,
 4.115953130712788]

In [145]:
dfOrig = pd.read_csv(fileData)
scaler = obterScaler(fileScaler)
dfOrig

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,162.856596,-1.619996e+05,-2.556977e+05,-0.000007,0.0,-0.000021,1.831071,-0.000117,0.918422,-0.036518,...,0.037389,0.021567,0.000615,-0.000374,-0.000113,-0.000041,-0.433582,0.003064,1,0
1,5601.013191,-1.017074e+05,2.603796e+06,-0.000007,0.0,-0.000021,-0.017121,0.964415,0.918422,13.818322,...,0.037389,0.021567,0.000615,-0.000374,-0.000113,-0.000041,-0.433582,0.003064,1,0
2,85.168645,-1.620350e+05,-2.551546e+05,-0.000007,0.0,-0.000021,1.831071,0.964415,0.918422,-0.036518,...,0.037389,0.021567,0.000615,-0.000374,-0.000113,-0.000041,-0.433582,0.003064,1,0
3,162.856596,-1.068854e+05,2.523255e+06,-0.000007,0.0,1.802761,0.906975,-0.000117,0.918422,23.054882,...,0.037389,0.021567,0.000615,-0.000374,-0.000113,-0.000041,-0.433582,0.003064,1,0
4,27.529197,-1.625670e+05,-3.055893e+05,-0.000007,0.0,-0.000021,-0.017121,-0.000117,-0.158884,-0.036518,...,0.037389,0.021567,0.000615,-0.000374,-0.000113,-0.000041,-0.433582,0.003064,1,0
5,75.144393,-1.623719e+05,-2.552321e+05,-0.000007,0.0,-0.000021,0.906975,-0.000117,0.918422,-0.036518,...,0.037389,0.021567,0.000615,-0.000374,-0.000113,-0.000041,-0.433582,0.003064,1,0
6,92.686834,-1.618577e+05,-1.844684e+05,-0.000007,0.0,-0.000021,6.451552,-0.000117,0.918422,0.887138,...,0.037389,0.021567,0.000615,-0.000374,-0.000113,-0.000041,-0.433582,0.003064,1,0
7,105.217149,-1.612725e+05,-2.636121e+05,-0.000007,0.0,-0.000021,2.755168,-0.000117,0.918422,-0.036518,...,0.037389,0.021567,0.000615,-0.000374,-0.000113,-0.000041,-0.433582,0.003064,1,0
8,27.529197,-1.254518e+05,-3.055893e+05,-0.000007,0.0,-0.000021,-0.017121,-0.000117,-0.158884,-0.036518,...,0.037389,0.021567,0.000615,-0.000374,-0.000113,-0.000041,-0.433582,0.003064,1,0
9,210.471792,-9.828489e+04,1.667029e+06,-0.000007,0.0,1.802761,5.527456,-0.000117,0.918422,-0.036518,...,0.037389,0.021567,0.000615,-0.000374,-0.000113,-0.000041,-0.433582,0.003064,1,0


In [146]:
dfResNorm = normalizar(dfRes, scaler)

In [147]:
dfOrigNorm = normalizar(dfOrig, scaler)

In [148]:
distancias(dfResNorm, dfOrigNorm)

[4.711889181771085,
 21.084490582146802,
 32.28368187591611,
 13.49319768915875,
 22.746889404525362,
 32.50599954859285,
 5.847631398182006,
 28.84125757173049,
 23.210988658826306,
 15.477325064274515,
 4.711888115724886,
 26.166012416679166,
 2.7964396081683915,
 4.003263025295861,
 29.706818993696483,
 20.703295226390424,
 9.57274242183931,
 22.157421161026246,
 16.836986811628837,
 3.243903657516356,
 3.8772583918840557,
 25.498003077306812,
 21.350807877902554,
 22.525519821506027,
 3.3714947846103653]

In [149]:
dfDif = diferenca(dfOrig, dfRes)
dfDif

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,0.143404,-0.445630,-0.299389,0.000007,0.0,0.000021,1.168929,0.000117,0.081578,2.036518,...,0.962611,-0.021567,-0.000615,0.000374,0.000113,0.000041,0.433582,-0.003064,0,1
1,-0.013191,0.357652,0.138515,0.000007,0.0,0.000021,3.017121,0.035585,0.081578,0.181678,...,0.962611,-0.021567,-0.000615,0.000374,0.000113,0.000041,0.433582,-0.003064,0,1
2,-0.168645,0.020368,-0.442550,0.000007,-0.0,0.000021,1.168929,0.035585,0.081578,3.036518,...,0.962611,-0.021567,-0.000615,0.000374,0.000113,0.000041,0.433582,-0.003064,0,1
3,0.143404,0.393370,-0.489807,0.000007,0.0,0.197239,2.093025,0.000117,0.081578,-0.054882,...,0.962611,-0.021567,-0.000615,0.000374,0.000113,0.000041,0.433582,-0.003064,0,1
4,3.470803,0.010339,0.279599,0.000007,0.0,0.000021,3.017121,0.000117,1.158884,1.036518,...,0.962611,-0.021567,-0.000615,0.000374,0.000113,0.000041,0.433582,-0.003064,0,1
5,-0.144393,-0.052651,0.149330,0.000007,-0.0,0.000021,2.093025,0.000117,0.081578,1.036518,...,0.962611,-0.021567,-0.000615,0.000374,0.000113,0.000041,0.433582,-0.003064,0,1
6,0.313166,-0.309623,0.354561,0.000007,0.0,0.000021,-3.451552,0.000117,0.081578,0.112862,...,0.962611,-0.021567,-0.000615,0.000374,0.000113,0.000041,0.433582,-0.003064,0,1
7,-0.217149,-0.498591,0.072395,0.000007,0.0,0.000021,0.244832,0.000117,0.081578,1.036518,...,0.962611,-0.021567,-0.000615,0.000374,0.000113,0.000041,0.433582,-0.003064,0,1
8,2.470803,-0.156641,0.279599,0.000007,0.0,0.000021,3.017121,0.000117,1.158884,1.036518,...,0.962611,-0.021567,-0.000615,0.000374,0.000113,0.000041,0.433582,-0.003064,0,1
9,-0.471792,-0.111162,-0.091475,0.000007,0.0,0.197239,-2.527456,0.000117,0.081578,1.036518,...,0.962611,-0.021567,-0.000615,0.000374,0.000113,0.000041,0.433582,-0.003064,0,1


In [150]:
#exportando datasets
dfOrig.to_csv(filepathRes + 'orig.csv', index=False)
dfOrigNorm.to_csv(filepathRes + 'orig_normalizado.csv', index=False)
dfRes.to_csv(filepathRes + 'advers.csv', index=False)
dfResNorm.to_csv(filepathRes + 'advers_normalizado.csv', index=False)
dfDif.to_csv(filepathRes + 'diferencas.csv', index=False)